# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:


def expected_return(P, rf, mkt_rp):
    list_weights = []
    total_invested = sum(P["Amount Invested"])
    for i in range(0, len(P["Amount Invested"])):
        list_weights.append(P["Amount Invested"][i] / total_invested)
    P_beta = 0
    for i in range(0, len(list_weights)):
        P_beta += list_weights[i] * P["Beta"][i]
    return P_beta * mkt_rp + rf
expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def listproduct(lst):
    x = 1
    for i in lst:
        x *= i
    return x

def volatility(P, cor):
    list_weights = []
    total_invested = sum(P["Amount Invested"])
    for i in range(0, len(P["Amount Invested"])):
        list_weights.append(P["Amount Invested"][i] / total_invested)
    P_Var = 0
    vrs = []
    betas = []
    for i in range(0, len(list_weights)):
        P_Var += ((list_weights[i] ** 2) * (P["Volatility"][i] ** 2))
    return (P_Var + (len(list_weights) * cor * listproduct(P["Volatility"]) * listproduct(list_weights))) ** 0.5
volatility(portfolio, -0.2)

0.24083189157584595

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    P_weight = vol/mkt_vol
    rf_weight = 1 - P_weight
    return rf + P_weight * mkt_rp
expected_return2(0.02, 0.05, 0.16, 0.2408)

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [211]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    
    ## Merge return_df and factors_df on month and year
    return_data = pd.read_csv(return_path)
    factors_data = pd.read_csv(factors_path)
    factors_data['year'] = factors_data.date // 100
    factors_data['month'] = factors_data.date % 100
    factors_data = factors_data.drop(columns=['date'])
    output = factors_data.merge(return_data)
    
    ##Seperate the dataframe by CUSIP and calculate each CUSIP's beta adding that as a column to the df
    ##Then we create a list of df's holding each CUSIP's df
    ll = []    
    output['diff'] = output['RET'] - output['RF']/100
    for group in output.groupby(by=["CUSIP"]):
        group[1]['beta'] = np.cov(group[1]['diff'], group[1]['Mkt-RF']/100 )[0,1]/ np.var(group[1]['Mkt-RF']/100)
        ll.append(group[1])

    ## Drop unnessecary columns and condense the df to have 1 row per CUSIP, by averaging the results of 
    ## numerical columns with that CUSIP
    final_df = pd.concat(ll,axis=0)
    final_df = final_df.drop(columns=['year','month'])
    final_df = final_df.groupby(by=["CUSIP"]).mean()
    final_df['cusip'] = final_df.index
    final_df = final_df.reset_index()
    
    x = pd.DataFrame(data = {
        "CUSIP" : final_df.cusip,
        "EXP_RETURN" : final_df.RET,
        "BETA" : final_df.beta,
    })
    return x
zz = CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")
zz


CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  2.603590
1     00108M10    0.011101  0.742466
2     00130H10    0.016152  1.460965
3     00163U10    0.015538  0.977903
4     00182C10    0.014772  1.412225
...        ...         ...       ...
3220   9959910    0.009933  0.765642
3221   9972410    0.012866  1.284197
3222   9972510    0.017177  1.070395
3223   9984910    0.009006  1.884472
3224   9985510    0.015699  1.171598

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [212]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.

    return np.polyfit(data.BETA, data.EXP_RETURN,deg=1)[1]
slope(zz)

0.007986271615212912